In [ ]:
'''
제거할 데이터(.xlsx) 제거
'''
import pandas as pd
import openpyxl
import os

# 엑셀 파일 경로
excel_path = "C:/Users/konyang/Desktop/project2025/data/npy_파일목록.xlsx"  # 파일 경로 수정

# 엑셀 로드
df = pd.read_excel(excel_path)

# '범위 밖' 열에서 유효한 파일명 추출
out_of_range_files = df['범위 밖'].dropna().unique().tolist()

print("🗑️ 삭제 대상 파일 목록:")
for fname in out_of_range_files:
    print(f" - {fname}")

# 삭제할 폴더 경로
folder_path = "C:/Users/konyang/Desktop/정규/결절 모델링 npy"

# 실제 삭제
for fname in out_of_range_files:
    file_path = os.path.join(folder_path, fname)
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"✅ 삭제됨: {file_path}")
    else:
        print(f"❌ 파일 없음: {file_path}")


In [29]:
'''
폴더 내에 있는 모든 파일을 다른 폴더로 옮김
'''
import os
import shutil

# A 폴더와 B 폴더 경로 설정
source_folder = 'C:/Users/konyang/Desktop/MeshCNN_TF/data/train/label'  # A 폴더 경로
destination_folder = 'C:/Users/konyang/Desktop/MeshCNN_TF/data/dataset/lebel'  # B 폴더 경로

# B 폴더가 없으면 생성
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# A 폴더의 모든 파일을 B 폴더로 이동
for filename in os.listdir(source_folder):
    source_file = os.path.join(source_folder, filename)
    destination_file = os.path.join(destination_folder, filename)

    if os.path.isfile(source_file):  # 파일만 이동
        shutil.move(source_file, destination_file)
        print(f"Moved: {filename}")


Moved: A0083_abnormal_label.npy
Moved: A0083_normal_label.npy
Moved: A0087_abnormal_label.npy
Moved: A0087_normal_label.npy
Moved: A0103_abnormal_label.npy
Moved: A0103_normal_label.npy
Moved: A0106_abnormal_label.npy
Moved: A0106_normal_label.npy
Moved: A0117_abnormal_label.npy
Moved: A0117_normal_label.npy
Moved: A0126_abnormal_label.npy
Moved: A0126_normal_label.npy
Moved: A0165_abnormal_label.npy
Moved: A0165_normal_label.npy
Moved: A0168_abnormal_label.npy
Moved: A0168_normal_label.npy
Moved: A0169_abnormal_label.npy
Moved: A0169_normal_label.npy
Moved: A0173_abnormal_label.npy
Moved: A0173_normal_label.npy
Moved: A0174_abnormal_label.npy
Moved: A0174_normal_label.npy
Moved: A0176_abnormal_label.npy
Moved: A0176_normal_label.npy
Moved: A0177_abnormal_label.npy
Moved: A0177_normal_label.npy
Moved: A0180_abnormal_label.npy
Moved: A0180_normal_label.npy
Moved: A0181_abnormal_label.npy
Moved: A0181_normal_label.npy
Moved: A0185_abnormal_label.npy
Moved: A0185_normal_label.npy
Moved: A

In [24]:
'''
폴더 내에 존재하는 모든 파일을 제거함
'''
import os
import glob

def clear_folder(folder_path):
    files = glob.glob(os.path.join(folder_path, "*"))
    deleted_count = 0

    for f in files:
        if os.path.isfile(f):
            os.remove(f)
            deleted_count += 1

    print(f"🧹 삭제 완료: {deleted_count}개 파일이 삭제되었습니다")

# 실행
clear_folder("C:/Users/konyang/Desktop/MeshCNN_TF/data/train/mesh_resized")


🧹 삭제 완료: 36개 파일이 삭제되었습니다


In [13]:
'''
npy 파일 형식을 obj 파일로 변환
'''

import os
import numpy as np
from skimage import measure
import trimesh

def combine_masks_by_filename(lung_folder, nodule_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    lung_files = [f for f in os.listdir(lung_folder) if f.endswith(".npy")]

    for fname in lung_files:
        lung_path = os.path.join(lung_folder, fname)
        nodule_path = os.path.join(nodule_folder, fname)
        base_name = os.path.splitext(fname)[0]
        out_path = os.path.join(output_folder, base_name + "_abnormal.obj")

        if not os.path.exists(nodule_path):
            print(f"⚠️ 누락: {nodule_path}")
            continue

        try:
            lung = np.load(lung_path)
            nodule = np.load(nodule_path)

            if lung.shape != nodule.shape:
                print(f"❌ shape 불일치: {fname}")
                continue

            combined = np.logical_or(lung, nodule).astype(np.uint8)
            verts, faces, normals, _ = measure.marching_cubes(combined, level=0.5)
            mesh = trimesh.Trimesh(vertices=verts, faces=faces, process=False)
            mesh.export(out_path)

            print(f"✅ 저장 완료: {out_path} ({verts.shape[0]} verts, {faces.shape[0]} faces)")

        except Exception as e:
            print(f"❌ 실패 [{fname}]: {e}")

"""
lung_folder: 폐 마스크 .npy 폴더
nodule_folder: 결절 마스크 .npy 폴더 (같은 파일명 존재)
output_folder: 결합된 .obj 저장 폴더
"""
combine_masks_by_filename(
    lung_folder="C:/Users/konyang/Desktop/data(진짜임)/혈관 모델링 npy",
    nodule_folder="C:/Users/konyang/Desktop/data(진짜임)/결절 모델링 npy",
    output_folder="C:/Users/konyang/Desktop/data(진짜임)/obj"
)

✅ 저장 완료: C:/Users/konyang/Desktop/data(진짜임)/obj\A0083_abnormal.obj (185292 verts, 359636 faces)
✅ 저장 완료: C:/Users/konyang/Desktop/data(진짜임)/obj\A0087_abnormal.obj (247141 verts, 476276 faces)
✅ 저장 완료: C:/Users/konyang/Desktop/data(진짜임)/obj\A0103_abnormal.obj (258894 verts, 495720 faces)
✅ 저장 완료: C:/Users/konyang/Desktop/data(진짜임)/obj\A0106_abnormal.obj (226659 verts, 441328 faces)
✅ 저장 완료: C:/Users/konyang/Desktop/data(진짜임)/obj\A0117_abnormal.obj (188417 verts, 368882 faces)
✅ 저장 완료: C:/Users/konyang/Desktop/data(진짜임)/obj\A0126_abnormal.obj (255059 verts, 491788 faces)
✅ 저장 완료: C:/Users/konyang/Desktop/data(진짜임)/obj\A0165_abnormal.obj (190264 verts, 371540 faces)
✅ 저장 완료: C:/Users/konyang/Desktop/data(진짜임)/obj\A0168_abnormal.obj (196350 verts, 382146 faces)
✅ 저장 완료: C:/Users/konyang/Desktop/data(진짜임)/obj\A0169_abnormal.obj (228380 verts, 443018 faces)
✅ 저장 완료: C:/Users/konyang/Desktop/data(진짜임)/obj\A0173_abnormal.obj (272352 verts, 524578 faces)
✅ 저장 완료: C:/Users/konyang/Desktop/data(진

In [35]:
import os
import shutil
import random

def split_dataset_files(source_dir, target_root, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42, ext=".obj"):
    random.seed(seed)

    # 1. 전체 파일 수집 (확장자 필터링 포함)
    all_files = [f for f in os.listdir(source_dir)
                 if os.path.isfile(os.path.join(source_dir, f)) and f.endswith(ext)]
    total = len(all_files)
    random.shuffle(all_files)

    # 2. 비율에 따른 분할
    train_count = int(total * train_ratio)
    val_count = int(total * val_ratio)
    test_count = total - train_count - val_count

    train_files = all_files[:train_count]
    val_files = all_files[train_count:train_count + val_count]
    test_files = all_files[train_count + val_count:]

    print(f"총 파일: {total}")
    print(f"➡️ train: {len(train_files)}, val: {len(val_files)}, test: {len(test_files)}")

    # 3. 분할 디렉토리 생성 및 파일 이동
    for split, file_list in zip(['train', 'val', 'test'], [train_files, val_files, test_files]):
        split_dir = os.path.join(target_root, split)
        os.makedirs(split_dir, exist_ok=True)

        for filename in file_list:
            src = os.path.join(source_dir, filename)
            dst = os.path.join(split_dir, filename)
            if not os.path.exists(dst):
                shutil.move(src, dst)
                print(f"📦 이동: {filename} → {split}/")

# -----------------------
# 실행 예시
# -----------------------
if __name__ == "__main__":
    split_dataset_files(
        source_dir=r"C:\Users\konyang\Desktop\MeshCNN_TF\data\dataset\simplified_label",
        target_root=r"C:\Users\konyang\Desktop\MeshCNN_TF\data\dataset\simplified_label",
        ext=".npy"
    )


총 파일: 350
➡️ train: 244, val: 70, test: 36
📦 이동: F010_normal_label.npy → train/
📦 이동: G0051_normal_label.npy → train/
📦 이동: G0041_normal_label.npy → train/
📦 이동: A0217_normal_label.npy → train/
📦 이동: A0241_abnormal_label.npy → train/
📦 이동: Z020_normal_label.npy → train/
📦 이동: Z080_normal_label.npy → train/
📦 이동: G0048_normal_label.npy → train/
📦 이동: Z031_normal_label.npy → train/
📦 이동: F021_normal_label.npy → train/
📦 이동: F025_abnormal_label.npy → train/
📦 이동: F028_normal_label.npy → train/
📦 이동: Z049_abnormal_label.npy → train/
📦 이동: F036_normal_label.npy → train/
📦 이동: Z054_abnormal_label.npy → train/
📦 이동: A0248_abnormal_label.npy → train/
📦 이동: A0205_normal_label.npy → train/
📦 이동: A0180_abnormal_label.npy → train/
📦 이동: Z012_abnormal_label.npy → train/
📦 이동: A0177_normal_label.npy → train/
📦 이동: A0248_normal_label.npy → train/
📦 이동: G0043_normal_label.npy → train/
📦 이동: A0227_abnormal_label.npy → train/
📦 이동: Z052_abnormal_label.npy → train/
📦 이동: F025_normal_label.npy → train/
📦 

In [41]:
import os
import glob
import pandas as pd

'''
확장자와 "_label"을 제거하여 파일 이름이 매칭되는지 확인하는 코드
'''

# 폴더 경로
vessel_folder = r"C:\Users\konyang\Desktop\MeshCNN_TF\data\dataset\simplified_mesh\test"
nodule_folder = r"C:\Users\konyang\Desktop\MeshCNN_TF\data\dataset\simplified_label\test"

# 파일 경로 가져오기
vessel_paths = glob.glob(os.path.join(vessel_folder, "*.obj"))
nodule_paths = glob.glob(os.path.join(nodule_folder, "*.npy"))

# .obj → 확장자 제거
vessel_files = [os.path.splitext(os.path.basename(f))[0] for f in vessel_paths]

# .npy → 확장자 및 "_label" 제거
nodule_files = [
    os.path.splitext(os.path.basename(f))[0].replace("_label", "")
    for f in nodule_paths
]

# 집합 비교
vessel_set = set(vessel_files)
nodule_set = set(nodule_files)

only_in_vessel = sorted(vessel_set - nodule_set)
only_in_nodule = sorted(nodule_set - vessel_set)

# 결과 정리
max_len = max(len(only_in_vessel), len(only_in_nodule))
df = pd.DataFrame({
    "혈관 폴더에만 있는 파일": only_in_vessel + [""] * (max_len - len(only_in_vessel)),
    "결절 폴더에만 있는 파일": only_in_nodule + [""] * (max_len - len(only_in_nodule))
})

# 개수 출력
print(f"📦 혈관 파일 (.obj): {len(vessel_files)}개")
print(f"📦 결절 파일 (.npy): {len(nodule_files)}개 (※ '_label' 제거 후 비교)")

# 출력
print("📂 두 폴더 간 매칭되지 않는 파일:")
print(df.to_string(index=False))


📦 혈관 파일 (.obj): 36개
📦 결절 파일 (.npy): 36개 (※ '_label' 제거 후 비교)
📂 두 폴더 간 매칭되지 않는 파일:
Empty DataFrame
Columns: [혈관 폴더에만 있는 파일, 결절 폴더에만 있는 파일]
Index: []
